# Tutorial: Serializing LLM Pipelines

- **Level**: Beginner
- **Time to complete**: 10 minutes
- **Components Used**: [`HuggingFaceLocalGenerator`](https://docs.haystack.deepset.ai/docs/huggingfacelocalgenerator), [`PromptBuilder`](https://docs.haystack.deepset.ai/docs/promptbuilder)
- **Prerequisites**: None
- **Goal**: After completing this tutorial, you'll understand how to serialize and deserialize between YAML and Python code.

> This tutorial uses Haystack 2.0. To learn more, read the [Haystack 2.0 announcement](https://haystack.deepset.ai/blog/haystack-2-release) or visit the [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/docs/intro).

## Overview

**📚 Useful Documentation:** [Serialization](https://docs.haystack.deepset.ai/docs/serialization)

Serialization means converting a pipeline to a format that you can save on your disk and load later. It's especially useful because a serialized pipeline can be saved on disk or a database, get sent over a network and more.

Although it's possible to serialize into other formats too, Haystack supports YAML out of the box to make it easy for humans to make changes without the need to go back and forth with Python code. In this tutorial, we will create a very simple pipeline in Python code, serialize it into YAML, make changes to it, and deserialize it back into a Haystack `Pipeline`.

## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/logging)

## Installing Haystack

Install Haystack 2.0 with `pip`:

In [1]:
# %%bash

# pip install haystack-ai

### Enabling Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/enabling-telemetry) for more details.

In [2]:
# from haystack.telemetry import tutorial_running

# tutorial_running(29)

## Creating a Simple Pipeline

First, let's create a very simple pipeline that expects a `topic` from the user, and generates a summary about the topic with `google/flan-t5-large`. Feel free to modify the pipeline as you wish. Note that in this pipeline we are using a local model that we're getting from Hugging Face. We're using a relatively small, open-source LLM.

In [3]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

template = """
Please create a summary about the following topic:
{{ topic }}
"""
builder = PromptBuilder(template=template)
llm = HuggingFaceLocalGenerator(
    model="google/flan-t5-large", task="text2text-generation", generation_kwargs={"max_new_tokens": 150}
)

pipeline = Pipeline()
pipeline.add_component(name="builder", instance=builder)
pipeline.add_component(name="llm", instance=llm)

pipeline.connect("builder", "llm")

/Users/mdashikadnan/Documents/adnanedu/python/Haystack/genai-haystack/genai-haystack/haystack_env/lib/python3.12/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(
/Users/mdashikadnan/Documents/adnanedu/python/Haystack/genai-haystack/genai-haystack/haystack_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚅 Components
  - builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - builder.prompt -> llm.prompt (str)

In [5]:
topic = "Climate change"
result = pipeline.run(data={"builder": {"topic": topic}})
print(result["llm"]["replies"][0])

/Users/mdashikadnan/Documents/adnanedu/python/Haystack/genai-haystack/genai-haystack/haystack_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Climate change is a major threat to the planet.


## Serialize the Pipeline to YAML

Out of the box, Haystack supports YAML. Use `dumps()` to convert the pipeline to YAML:

In [6]:
yaml_pipeline = pipeline.dumps()

print(yaml_pipeline)

components:
  builder:
    init_parameters:
      required_variables: null
      template: '

        Please create a summary about the following topic:

        {{ topic }}

        '
      variables: null
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: mps
        model: google/flan-t5-large
        task: text2text-generation
      stop_words: null
      streaming_callback: null
      token:
        env_vars:
        - HF_API_TOKEN
        - HF_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}



You should get a pipeline YAML that looks like the following:

```yaml
components:
  builder:
    init_parameters:
      template: "\nPlease create a summary about the following topic: \n{{ topic }}\n"
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cpu
        model: google/flan-t5-large
        task: text2text-generation
        token: null
      stop_words: null
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}

```

## Editing a Pipeline in YAML

Let's see how we can make changes to serialized pipelines. For example, below, let's modify the promptbuilder's template to translate provided `sentence` to French:

In [8]:
yaml_pipeline = """
components:
  builder:
    init_parameters:
      template: "\nPlease translate the following to French: \n{{ sentence }}\n"
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cpu
        model: google/flan-t5-large
        task: text2text-generation
        token: null
      stop_words: null
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}
"""

## Deseriazling a YAML Pipeline back to Python

You can deserialize a pipeline by calling `loads()`. Below, we're deserializing our edited `yaml_pipeline`:

In [9]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

new_pipeline = Pipeline.loads(yaml_pipeline)

Now we can run the new pipeline we defined in YAML. We had changed it so that the `PromptBuilder` expects a `sentence` and translates the sentence to French:

In [10]:
new_pipeline.run(data={"builder": {"sentence": "I love capybaras"}})

{'llm': {'replies': ['Je me félicite des capybaras !']}}

## What's next

🎉 Congratulations! You've serialzed a pipeline into YAML, edited it and ran it again!

If you liked this tutorial, you may also enjoy:
-  [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates). Thanks for reading!